# 経営情報各論BⅡ：8. 検定
## (1)推定
### この章でやること

母数，すなわち母集団のパラメタをこれから推定する．
母集団分布に正規分布を仮定した場合、母数がわかれば母集団分布が推定できることになる。この 章では、まずは単純な点推定を説明したのち、推定誤差を加味した区間推定の方法を説明する。

### 推定のための用語
|言葉|意味|例と表記|
|---|-----------|-----------------------------|
|統計量|標本を得ることで値が<br>確定する確率変数|平均値，分散，中央値，<br>最頻値|
|母数|母集団の持つ特性値|母平均，母分散|
|推定量|母数$\theta$を推定するために<br>用いる統計量$\hat{\theta}$|母平均$\mu$に対する標本平均$\bar{X}$，<br>母分散$\sigma^2$に対する不偏分散$s^2$|
|推定値|標本から得られた<br>推定量の値|平均値$\bar{x}$，分散$s^2$の値|
|母集団分布|母集団の個体の分布|正規分布|
|標本分布|推定量の分布|正規分布，2項分布，$t$分布|

20才男子の身長を例にして，これらの用語を確認する．

![図DS2-testing-1-1](https://drive.google.com/uc?id=1-AEg2FWMToG52LHSCg8lwRunSR7M9YND)

### 分析の準備

まずは、必要なライブラリの読み込みや表示桁数の設定などを行う。

In [1]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# グラフを描画するライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# 表示桁数の指定
%precision 3
# グラフをjupyter Notebook内に表示させるための指定
%matplotlib inline

続いて、今回の分析の対象となるデータを読み込む。魚の体長を測定した架空のデータ。サンプルサイズは10。シリーズ形式で読み込む。

In [2]:
# データの読み込み
fish = pd.read_csv("ds2-testing-1-fish_length.csv")["length"]
fish

0    4.352982
1    3.735304
2    5.944617
3    3.798326
4    4.087688
5    5.265985
6    3.272614
7    3.526691
8    4.150083
9    3.736104
Name: length, dtype: float64

### 点推定
点推定とは、母数（母集団分布のパラメタ）をある1つの値として指定する推定方法。母平均を推定する場合は、標本平均を推定量として使う。要するに、私たちがやることは、標本から平均値を計算するだけ。これで推定が完了となる。なんだかとても簡単なのですが、標本平均を使ってもよい理由があることを思い出してください。標本平均は不偏性も一致性も満たす統計量です。望ましい性質を持つ統計量であるため、母平均の推定値としては標本平均が使われるということです。

Pythonを使って、母平均の点推定を実装する。標本平均を計算するだけ。

In [3]:
# 母平均の点推定
mu = np.mean(fish)
mu

4.187

標本平均が4.187でしたので、母平均も4.187だろうと推定します。これが点推定です。今回は実用上多く使われる母平均の推定の方法を中心に説明します。しかし、母分散の推定に関しても同じように標本から計算された統計量を使用できます。母分散の点推定値としては、不偏分散を使います。

In [4]:
# 母分散の点推定
sigma_2 = np.var(fish, ddof = 1)
sigma_2

0.680

### 区間推定
**区間推定**とは、推定値に幅を持たせた推定方法のことです。推定値の幅の計算には、確率の考え方を用います。幅を持たせるこ
とで、推定誤差を加味することができます。推定誤差が小さければ、区間推定の幅は狭くなります。サンプルサイズが大きくても、やはり区間推定の幅は狭くなります。
### 点推定と区間推定
|推定法|意味|
|-------|-----------------------------------|
|点推定|母集団に関する真の値を，ある１つの推定値で予想する方法|
|区間推定|例えば「95%の確率で，この区間に母数が入る」というような推定法．<br>ある確率を与えて，母集団に関する真の値が入る区間を提示する推定法|

### 信頼係数・信頼区間
**信頼係数**とは、区間推定の幅における信頼の度合いを、確率で表現したものです。例えば95%や99%といった数値がよく使われます。**信頼区間**とは、ある信頼係数を満たす区間のことです。同じデータを対象とした場合は、信頼係数が大きいほど、信頼区間の幅は広くなります。信頼の度合いを上げようと思うと、どうしても安全第一で幅を広く取らざるを得ないということです。
### 信頼限界
**信頼限界**とは、信頼区間の下限値・上限値のことです。各々，**下側信頼限界**，**上側信頼限界**とも呼ばれます．
### 信頼区間の計算方法
「（標本平均-母平均）÷標準誤差」で計算されるt値はt分布に従うことを2.分布で説明しました。区間推定の際は、t分布のパーセント点（ある確率になる基準値）を用います。信頼係数を95%とするならば、t分布における、2.5%点と97.5%点を計算します。t分布に従う変数がこの区間に入る確率は95%ということになります。この区間を使えばよいのです。

### 実装：区間推定
区間推定に必要となる情報は、自由度（サンプルサイズ-1）、標本平均、標準誤差の3つです。標本平均はすでに計算済みですので、残りの2つを計算します。まずは自由度です。自由度という名前は気にせず、単にサンプルサイズから1を引けばよいです。サンプルサイズが大きいと推定精度が上がります。なのでサンプルサイズの情報が必要だということです。

In [5]:
# 自由度
df = len(fish) - 1
df

9

次は標準誤差を求めます。standard errorの略でseという変数名にしました。

In [6]:
# 標準誤差
sigma = np.std(fish, ddof = 1)
se = sigma / np.lib.scimath.sqrt(len(fish))
se

0.261

あとはこれらの情報を使って信頼区間を計算します。`stats. t. interval`関数を使います。引数には信頼係数`alpha`、自由度`df`、標本平均`loc`、標準誤差`scale`を指定します。出力の1つ目が下側信頼限界、2番目の要素が上側信頼限界となります。

In [7]:
# 区間推定
interval = stats.t.interval(
#    alpha = 0.95, df = df, loc = mu, scale = se)
    confidence = 0.95, df = df, loc = mu, scale = se)
interval

(3.597, 4.777)

95%信頼区間は3.597から4.777となりました。

### 信頼区間の求め方の詳細
Pythonを使えば簡単に計算が終わるのですが、興味のある方のために、信頼区間の計算式を紹介しておきます。ただしこの計算を今後繰り返すことはありません。以下の手順で信頼区間を計算します。

1. ある自由度を持つ$t$分布における、97.5%点を計算する
    - $t$分布における、97.5%点を$t_{0.975}$と表記する
    - $t$分布は平均に対して左右対称であるため、2.5%点は$-t_{0.975}$として計算できる
    - $t$分布に従う変数が$-t_{0.975}$以上$t_{0.975}$以下になる確率は95%である
        - この時の95%が信頼係数となる
2. 『標本平均$\;- t_{0.975}\;\times\;$標準誤差』が下側信頼限界となる 
3. 『標本平均$\;+ t_{0.975}\;\times\;$標準誤差』が上側信頼 限界となる

数式を使って計算手順の意味を確認します。まずは、$t$値の計算式を思い出してください。ただし$\hat{\mu}$は標本平均、$\mu$は母平均、$\hat{se}$は標本から計算された標準誤差です。

$t値 = \frac{\hat{\mu}-\mu}{\hat{se}}$

$t$値が$-t_{0.975}$以上$t_{0.975}$以下になる確率は95%です。

$P\Bigl(-t_{0.975}\leq \frac{\hat{\mu}-\mu}{\hat{se}} \leq t_{0.975}\Bigr)=95\%$

これを母平均$\mu$について解くと、以下のようになります。

$P(\hat{\mu}-t_{0.975}\times\hat{se}\leq\mu\leq\hat{\mu}+t_{0.975}\times\hat{se})= 95\%$

Pythonで確認してみます。

まずは，97.5%点を求めます．

In [8]:
# 97.5%点
t_975 = stats.t.ppf(q = 0.975, df = df)
t_975

2.262

下側信頼限界は以下のように計算される．

In [9]:
# 下側信頼限界
lower = mu - t_975 * se
lower

3.597

上側信頼限界は以下のように計算される．

In [10]:
# 上側信頼限界
upper = mu + t_975 * se
upper

4.777

<h3>信頼区間の幅を決める要素</h3>
標本における分散が大きければ「データが平均値から離れている→平均値をあまり信頼できない」ことになるので、信頼区間の幅が広くなります。物は試しで、標本標準偏差を10倍に増やしてから95%信頼区間を計算してみましょう。かなり幅が広くなります。

In [11]:
# 標本標準偏差が大きいと、信頼区間は広くなる
se2 = (sigma*10) / np.lib.scimath.sqrt(len(fish))
stats.t.interval(
    confidence = 0.95, df = df, loc = mu, scale = se2)

(-1.713, 10.087)

信頼区間の幅が広いというのは「真の母平均がどこに位置しているのかがよくわからない」ということだと解釈すれば直感によく合う結果です。逆に、サンプルサイズが大きくなれば、標本平均を信頼できるようになるため、信頼 区間は狭くなります。これもPythonで計算します。サンプルサイズを10倍にしてみました。サンプルサイズが大きくなると、自由度が大きくなり、標準誤差が小さくなることに注意します。

In [12]:
# サンプルサイズが大きいと、信頼区間は狭くなる
df2 = (len(fish)*10) - 1
se3 = sigma / np.lib.scimath.sqrt(len(fish)*10)
stats.t.interval(
    confidence = 0.95, df = df2, loc = mu, scale = se3)

(4.023, 4.351)

まったく同一のデータであった場合には、信頼係数が大きいほど、安全を見込んで信頼区間の幅は広く取られます。99%信頼区間は以下のように計算されます。95%信頼区間よりも幅が広くなっていることに注目してください。

In [13]:
# 99%信頼区間
stats.t.interval(
    confidence = 0.99, df = df, loc = mu, scale = se)

(3.339, 5.035)

### 区間推定の結果の解釈

信頼係数95%における「95%」の意味を、今までは信頼の度合いというややあいまいな表現で濁していた．
この節では，シミュレーションを通して区間推定の結果の解釈を試みる．
信頼係数95%の「95%」は，以下のようにして得られる．
    
1. 真の母集団分布から標本を抽出する
2. 今回と同じやり方で95%信頼区間を計算する．
3. この試行をたくさん繰り返す
4. すべての試行のうち，「真の母数」が信頼区間に含まれている割合が95%

![図7-1](https://drive.google.com/uc?id=1-9PKpFSBtwIbrfYz0qWwZ-q5xQwtxOhL)

シミュレーションを通して確認する．
試行回数は20,000回とする．

信頼区間が母平均(4)を含んでいればTrueをとる変数を用意します．試行回数は20,000回ですので，要素数20,000のアレイになります．TrueかFalesしかとらないので「dtype="bool"」としておいた．

In [14]:
# 信頼区間が母平均(4)を含んでいればTrue
be_included_array = np.zeros(20000, dtype = "bool")
be_included_array

array([False, False, False, ..., False, False, False])

シミュレーションを実行します．

In [15]:
# 「データを10個選んで95%信頼区間を求める」試行を20000回繰り返す
# 信頼区間が母平均(4)を含んでいればTrue
np.random.seed(1)
norm_dist = stats.norm(loc = 4, scale = 0.8)
for i in range(0, 20000):
    sample = norm_dist.rvs(size = 10)
    df = len(sample) - 1
    mu = np.mean(sample)
    std = np.std(sample, ddof = 1)
    se = std / np.lib.scimath.sqrt(len(sample))
    interval = stats.t.interval(0.95, df, mu, se)
    if(interval[0] <= 4 and interval[1] >= 4):
        be_included_array[i] = True

信頼区間が母平均(4)を含んでいた割合を求める．およそ0.95となる．

In [16]:
sum(be_included_array) / len(be_included_array)

0.948